In [1]:
import ROOT as root
import os

def getall_filenames(*, startdir):
    # This func outputs a List of all Files in a given directory and its subdirectories
    f = []
    if startdir[-1] == "/":
        startdir = startdir[0:-1]
    w = os.walk(startdir)
    for (dirpath, dirnames, filenames) in w:
        for i in filenames:
            f.append(f"{dirpath}/{i}")
    return f

#load ROOT File and extract graph Data into a List with X, Y and Intex as tuple
def extract_graph_data(*, filename):
    file = root.TFile(f"{filename}" , "READ")
    file.ls()
    data = []
    graph = file.Get("IV_Scan")
    for i in range(graph.GetN()):
        x, y = graph.GetX()[i], graph.GetY()[i]
        data.append((i, x, y))
#        print("Point %d: x = %f, y = %f" % (i, graph.GetX()[i], graph.GetY()[i]))
    return data
    





ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!
Invoking:
  LC_ALL=C x86_64-conda-linux-gnu-c++  -O3 -DNDEBUG -xc++ -E -v /dev/null 2>&1 | sed -n -e '/^.include/,${' -e '/^ \/.*++/p' -e '}'
Results was:
With exit code 0


Welcome to JupyROOT 6.30/04


In [2]:
#import self defined packages
from ast import Pass
from math import sqrt
from cmd import IDENTCHARS
from colorsys import hls_to_rgb
import sys
from urllib.parse import ParseResultBytes

from matplotlib.colors import get_named_colors_mapping
sys.path.append("/home/tiancheng/git_repo/my_python_package")
from python_tools.plot_helper.IV_analysiser import IV_analysiser 
from python_tools.plot_helper.plot_helper import plot_helper 
from python_tools.print_helper import bcolors
from python_tools.plot_helper.pyroot_helper import root_helper
bc = bcolors()
ph = plot_helper()
IV_ana = IV_analysiser()
root_h  = root_helper()
# python packages
from pathlib import Path
import numpy as np
#import ROOT
import ROOT


runID_all = [3,5,1,1,6,7,7,4]

def cal_Rq(R_measured, Error_R_measured, R_constant, Error_R_Constant):
    '''
    calcualte R_quenching based on measured R
    R_measured: measured R
    Error_R_measured: error measured R in [ohm]
    R_Constant: R used for current limitation
    Error_R_Constant: error of R used in current limitation [ohm]
    '''
    N_pixels = 3584
    R_q = (R_measured-R_constant)*N_pixels
    R_q_err = sqrt(Error_R_measured**2+Error_R_Constant**2)*N_pixels
    return R_q, R_q_err

def get_all_forward_grs(dir_name,matrixID,ID_start,ID_stop,ID_step,hv_stop,hv_nsteps):
    grs=[]
    for runID in range(ID_start,ID_stop+1,ID_step):
        file_in_name = f'{dir_name}/IV_scan_run{runID}_matrix{matrixID}_UID5_forward_HV_start0mV_hv_stop{int(hv_stop*1000.)}mV_n_steps{hv_nsteps}_TDelay200ms.root'
        if not Path(file_in_name).is_file():
            print(f'{bc.ERROR}file {file_in_name} does NOT exist!!{bc.ENDC}')
            continue
        file_in_tmp = ROOT.TFile(file_in_name)
        gr_tmp = file_in_tmp.Get("IV_Scan")
        if(gr_tmp):
            gr_tmp = ph.NameIt(gr_tmp,name=f'IV_Scan_run{runID}',title=f'Run {runID}',xtitle='Forward voltage [V]',ytitle='Current [A]')
            grs.append(gr_tmp)
    return grs
def get_all_reverse_grs(dir_name,matrixID,ID_start,ID_stop,ID_step,hv_stop,hv_nsteps):
    grs=[]
    for runID in range(ID_start,ID_stop+1,ID_step):
        file_in_name = f'{dir_name}/IV_scan_run{runID}_matrix{matrixID}_UID5_reverse_HV_start0mV_hv_stop{int(hv_stop*1000.)}mV_n_steps{hv_nsteps}_TDelay200ms.root'
        if not Path(file_in_name).is_file():
            print(f'{bc.ERROR}file {file_in_name} does NOT exist!!{bc.ENDC}')
            continue
        file_in_tmp = ROOT.TFile(file_in_name)
        gr_tmp = file_in_tmp.Get("IV_Scan")
        if(gr_tmp):
            gr_tmp = ph.NameIt(gr_tmp,name=f'IV_Scan_run{runID}',title=f'Run {runID}',xtitle='HV [V]',ytitle='Current [A]')
            grs.append(gr_tmp)
    return grs
def get_obj(file_name, gr_name):
    if Path(file_name).is_file():
        file = ROOT.TFile(file_name)
        gr = file.Get(gr_name)
        file.Close()
        return gr
    else:
        print(bc.ERROR+f"file:{file_name} is not a file"+bc.ENDC)
        return 0
def read_T(file_in_name):
    if not Path(file_in_name).is_file():
        print(f'{bc.ERROR}file {file_in_name} does NOT exist!!{bc.ENDC}')
        return 
    file_in_tmp = ROOT.TFile(file_in_name)
    return file_in_tmp.Get("T_allRuns")

def fit_IV_3rd_dev(gr_tmp,Vbr_ref):
    '''
    '''
    #        0 ==> A: amplitude
    #        1 ==> sigma: std dev
    #        2 ==> V01: mean
    #        3 ==> h: hysteresis
    low = Vbr_ref-2.5
    high = Vbr_ref+2.5
    par_A_ref = 3.
    gr = ROOT.TGraph()
    for i in range(gr_tmp.GetN()):
        x = gr_tmp.GetX()[i]
        if x> low and x<high:
            gr.SetPoint(gr.GetN(),x,gr_tmp.GetY()[i]*1.e8)
    f_IV_3rd_dev = ROOT.TF1("f_IV_3rd_dev","[0]*(2-[3]*(x-[2])/pow([1],2))*exp(-pow(x-[2],2)/2/pow([1],2))",low,high)
    f_IV_3rd_dev.SetParNames("Amplitude","#sigma","V_{br}","hysteresis")
    f_IV_3rd_dev.SetParameter(0,par_A_ref)
    f_IV_3rd_dev.SetParLimits(0,0.1*par_A_ref,2.*par_A_ref)
    f_IV_3rd_dev.SetParameter(1,0.12)
    f_IV_3rd_dev.SetParLimits(1,0.1,0.3)
    f_IV_3rd_dev.SetParameter(2,Vbr_ref)
    f_IV_3rd_dev.SetParLimits(2,Vbr_ref-0.5,Vbr_ref+0.5 )
    f_IV_3rd_dev.SetParameter(3,0.5)
    f_IV_3rd_dev.SetParLimits(3,0.1,1.)
    fit_stat = gr.Fit(f_IV_3rd_dev,"QR")
    fit_stat = gr.Fit(f_IV_3rd_dev,"QR")
    if int(fit_stat)!=0:
        gr.Fit(f_IV_3rd_dev,"QR")
        print("do second fit")
    return np.asarray(f_IV_3rd_dev.GetParameters()),np.asarray(f_IV_3rd_dev.GetParErrors()),f_IV_3rd_dev.GetChisquare()/f_IV_3rd_dev.GetNDF(),gr


def do_ana_single_reverse(dir_in,matrixID):
    '''
    '''
    dir_name=dir_in + f"matrix{matrixID}/"
    if not Path(dir_name).is_dir():
        print(bc.ERROR+f"{dir_name} is not a folder"+bc.ENDC)
        exit()

    fout_name= "ana_IV_scan_reverse"
    Vbr_ref = 51.5
    gr_org_all=[]
    gr_ICorr_all=[]
    gr_OV_all=[]
    gr_ZeroCorr_all=[]
    gr_ZeroCorr_35V_all=[]
    gr_3rd_der_all=[]
    gr_3rd_der_fit_scale_all=[]
    gr_sigma = ROOT.TGraphErrors()
    gr_sigma = ph.NameIt(gr_sigma,name=f'sigma_in_diff_ch',title=f'sigma',xtitle='UID',ytitle='#sigma of 3rd derivative fit [V]')
    gr_Vbr = ROOT.TGraphErrors()
    gr_Vbr = ph.NameIt(gr_Vbr,name=f'Vbr_in_diff_ch',title=f'Breakdown Volatge along runs',xtitle='UID',ytitle='V_{br} [V]')
    gr_h = ROOT.TGraphErrors()
    gr_h = ph.NameIt(gr_h,name=f'h_in_diff_ch',title=f'Breakdown Volatge along runs',xtitle='UID',ytitle='Hysterisis [V]')
    
    gr_I_OV2V = ROOT.TGraph()
    gr_I_OV2V = ph.NameIt(gr_I_OV2V,name=f'I_with_OV2V',title=f'Current with OV=2V',xtitle='UID',ytitle='I_{dark} (OV = 2V) [mA]')
    gr_I_OV3V = ROOT.TGraph()
    gr_I_OV3V = ph.NameIt(gr_I_OV3V,name=f'I_with_OV3V',title=f'Current with OV=3V',xtitle='UID',ytitle='I_{dark} (OV = 3V) [mA]')
    gr_I_OV4V = ROOT.TGraph()
    gr_I_OV4V = ph.NameIt(gr_I_OV4V,name=f'I_with_OV4V',title=f'Current with OV=4V',xtitle='UID',ytitle='I_{dark} (OV = 4V) [mA]')
    gr_I_HV55V = ROOT.TGraph()
    gr_I_HV55V = ph.NameIt(gr_I_HV55V,name=f'I_with_HV55V',title=f'Current with HV=55V',xtitle='UID',ytitle='I_{dark} (HV = 55V) [mA]')
    
    for UID in range(16): # becareful this UID is start from 0
        file_name_tmp = dir_name + f'IV_scan_run{runID_all[matrixID]}_UID{UID}_reverse_HV_start0mV_hv_stop58000mV_n_steps291_TDelay500ms.root'
        if not Path(file_name_tmp).is_file():
            print(bc.ERROR+f"{file_name_tmp} is not a file"+bc.ENDC)
            continue
        file_tmp = ROOT.TFile(file_name_tmp)
        gr_tmp = file_tmp.Get("IV_Scan")
        gr_tmp= ph.NameIt(gr_tmp,name=f'IV_Scan_UID{UID}',title=f'IV Curve UID{UID}',xtitle='HV [V]',ytitle='Current [A]')
        gr_org_all.append(gr_tmp)
        gr_ZeroCorr = IV_ana.gr_ZeroCorr(gr_tmp)
        gr_ZeroCorr = ph.NameIt(gr_ZeroCorr,name=f'IV_Scan_ZeroCorr_UID{UID}',title=f'IV Curve after Zero Correction UID{UID}',xtitle='HV [V]',ytitle='Current [A]')
        gr_ZeroCorr_35V = IV_ana.gr_ZeroCorr(gr_tmp,35.)
        gr_ZeroCorr_35V = ph.NameIt(gr_ZeroCorr_35V,name=f'IV_Scan_ZeroCorr_35V_UID{UID}',title=f'IV Curve after Zero Correction UID{UID}',xtitle='HV [V]',ytitle='Current [A]')
        gr_ICorr = IV_ana.current_corr(gr_ZeroCorr)
        gr_ICorr = ph.NameIt(gr_ICorr,name=f'IV_Scan_ICorr_UID{UID}',title=f'IV Curve after Current Correction UID{UID} (R=1k#Omega)',xtitle='HV [V]',ytitle='Current [A]')
        gr_3rd_IV = root_h.get_gr_nth_derivative(gr_ZeroCorr,3)
        gr_3rd_IV = ph.NameIt(gr_3rd_IV,name=f'IV_curve_3rd_dev_UID{UID}',title=f'3rd derivative of IV Curve after Zero Correction UID{UID}',xtitle='HV [V]',ytitle='#frac{d^{3}I}{dV^{3}}')
        pars_Vbr ,par_Vbr_errors, chi2,gr_fit = fit_IV_3rd_dev(gr_3rd_IV,Vbr_ref)
        gr_fit = ph.NameIt(gr_fit,name=f'IV_curve_3rd_dev_UID{UID}_scaled_1e8',title=f'3rd derivative of IV Curve after Zero Correction UID{UID}',xtitle='HV [V]',ytitle='#frac{d^{3}I}{dV^{3}} [a.u.]')
        Vbr = pars_Vbr[2]
        h   = pars_Vbr[3]
        gr_OV = IV_ana.get_OV(gr_ICorr,Vbr)
        gr_OV = ph.NameIt(gr_OV,name=f'IV_Scan_OV_UID{UID}',title=f'IV Curve along Overvoltage UID{UID}',xtitle='Overvoltage [V]',ytitle='Current [A]')
        gr_I_OV2V.SetPoint(gr_I_OV2V.GetN(),UID,gr_OV.Eval(2.)*1000.) #! cahnge unit to mA
        gr_I_OV3V.SetPoint(gr_I_OV3V.GetN(),UID,gr_OV.Eval(3.)*1000.)
        gr_I_OV4V.SetPoint(gr_I_OV4V.GetN(),UID,gr_OV.Eval(4.)*1000.)
        gr_I_HV55V.SetPoint(gr_I_HV55V.GetN(),UID,gr_ICorr.Eval(55.)*1000.)

        gr_ICorr_all.append(gr_ICorr)
        gr_OV_all.append(gr_OV)
        gr_ZeroCorr_all.append(gr_ZeroCorr)
        gr_3rd_der_all.append(gr_3rd_IV)
        gr_3rd_der_fit_scale_all.append(gr_fit)
        gr_ZeroCorr_35V_all.append(gr_ZeroCorr_35V)

        gr_sigma.SetPoint(gr_sigma.GetN(),UID,pars_Vbr[1])
        gr_sigma.SetPointError(gr_sigma.GetN()-1,0.,par_Vbr_errors[1])
        gr_Vbr.SetPoint(gr_Vbr.GetN(),UID,pars_Vbr[2])
        gr_Vbr.SetPointError(gr_Vbr.GetN()-1,0.,par_Vbr_errors[2])
        gr_h.SetPoint(gr_h.GetN(),UID,pars_Vbr[3])
        gr_h.SetPointError(gr_h.GetN()-1,0.,par_Vbr_errors[3])
        file_tmp.Close()
        #TODO continue here
    #save datas
    file_out = ROOT.TFile(f"{dir_name}/{fout_name}.root","recreate")
    dir_IVplot = file_out.mkdir("IV_scan/")
    root_h.write_grs(gr_org_all,dir_IVplot)
    dir_ZeroCorr = file_out.mkdir("IV_scan_ZeroCorr/")
    root_h.write_grs(gr_ZeroCorr_all,dir_ZeroCorr)
    dir_ZeroCorr_35V = file_out.mkdir("IV_scan_ZeroCorr_35V/")
    root_h.write_grs(gr_ZeroCorr_35V_all,dir_ZeroCorr_35V)
    dir_ICorr= file_out.mkdir("IV_scan_ICorr/")
    root_h.write_grs(gr_ICorr_all,dir_ICorr)
    dir_3rd_dev= file_out.mkdir("IV_scan_3r_dev/")
    root_h.write_grs(gr_3rd_der_all,dir_3rd_dev)
    dir_3rd_dev_fit= file_out.mkdir("IV_scan_3r_dev_fit_scale/")
    root_h.write_grs(gr_3rd_der_fit_scale_all,dir_3rd_dev_fit)
    dir_OV= file_out.mkdir("IV_scan_OV/")
    root_h.write_grs(gr_OV_all,dir_OV)
    file_out.cd()
    gr_sigma.Write()
    gr_Vbr.Write()
    gr_h.Write()
    gr_I_OV2V.Write()
    gr_I_OV3V.Write()
    gr_I_OV4V.Write()
    gr_I_HV55V.Write()
    file_out.Close()
    print(f"done with matrix{matrixID}!!")
    return gr_sigma,gr_Vbr,gr_h,gr_I_OV2V,gr_I_OV3V,gr_I_OV4V,gr_I_HV55V

def do_ana_single_forward(dir_in,matrixID,stat_annealing):
    dir_name=dir_in + f"matrix{matrixID}_before_annealing"
    if stat_annealing==1:
        dir_name=dir_in + f"matrix{matrixID}_after_annealing"
    fout_name= "ana_IV_scan_forward"
    gr_T = read_T(f'{dir_name}/T_allRuns.root')
    runIDs = np.asarray(gr_T.GetX() ).astype(int)
    runID_start=runIDs[0]
    runID_stop =runIDs[-1]
    hv_stop=3  #diff from matrix to matrix
    N_pts = 31
    grs=get_all_forward_grs(dir_name,matrixID,runID_start,runID_stop,1,hv_stop,N_pts) #get all the grs
    print(bc.INFO+f'N of Runs: {len(grs)}'+bc.ENDC)

    gr_R    = ROOT.TGraphErrors()
    gr_R    = ph.NameIt(gr_R,name=f'gr_R',title=f'Resistor vs. T',xtitle='T [#circC]',ytitle='Resistor [#Omega]')
    h_R     = ROOT.TH1D("hist_R","Resistor",500,750.,1250)
    h_R     = ph.NameIt(h_R,ytitle='Entries',xtitle='R [#Omega]')
    gr_V_forward  = ROOT.TGraphErrors()
    gr_V_forward  = ph.NameIt(gr_V_forward,name=f'gr_V_forward',title=f'Forward voltage vs. T',xtitle='T [#circC]',ytitle='V_{f} [V]')
    h_V_forward    = ROOT.TH1D("hist_V_forward","Forward voltage",500,0.,1.)
    h_V_forward    = ph.NameIt(h_V_forward,title='Entries',xtitle='V_{f} [V]')
    gr_chi2  = ROOT.TGraphErrors()
    gr_chi2  = ph.NameIt(gr_chi2,name=f'gr_chi2',title=f'chi2 vs. T',xtitle='T [#circC]',ytitle='#chi^{2}/NDF')
    #* start analysis
    cnt=0
    grs_scale  = [] # scale the unit of current to mA
    scale_current = 1000.   # scale to mA
    for gr in grs:
        gr_tmp = ROOT.TGraph()
        gr_tmp = ph.NameIt(gr_tmp,name=f'I_V_curve_scale_{cnt}',title=f'I-V curve',xtitle='Voltage [V]',ytitle='Current [mA]')
        for i in range(gr.GetN()):
            gr_tmp.SetPoint(i,gr.GetX()[i],gr.GetY()[i]*scale_current)
        grs_scale.append(gr_tmp)
        #get T
        T_curr = gr_T.GetY()[cnt]
        cnt +=1
        #ana
        pars,parerrors,chi2 = IV_ana.lin_fit2(gr_tmp,-3.1,-1.5)
        #pars,parerrors,chi2 = IV_ana.lin_fit2(gr_tmp,-3.1,-1.,par_name0="R[k#Omega]",par_name1="V_{f}[V]")
        gr_R.SetPoint(gr_R.GetN(),T_curr,pars[0]*scale_current)
        gr_R.SetPointError(gr_R.GetN()-1,0.,parerrors[0]*scale_current)
        h_R.Fill(pars[0]*scale_current)
        gr_V_forward.SetPoint(gr_V_forward.GetN(),T_curr,pars[1])
        gr_V_forward.SetPointError(gr_V_forward.GetN()-1,0.,parerrors[1])
        h_V_forward.Fill(pars[1])
        gr_chi2.SetPoint(gr_chi2.GetN(),T_curr,chi2)

    #save datas
    file_out = ROOT.TFile(f"{dir_name}/{fout_name}.root","recreate")
    gr_R.Write()
    h_R.Write()
    gr_V_forward.Write()
    h_V_forward.Write()
    gr_chi2.Write()
    dir_IVplot_forward = file_out.mkdir("IV_scan_forward/")
    root_h.write_grs(grs,dir_IVplot_forward)
    dir_IVplot_forward_mA = file_out.mkdir("IV_scan_forward_mA/")
    root_h.write_grs(grs_scale,dir_IVplot_forward_mA)
    return gr_R, h_R, gr_V_forward, h_V_forward,gr_chi2, grs_scale

def do_reverse_IV_ana(dir_in,file_out_name):
    matrixID_all = np.array([1,2,3,4,5,6])
    f_out = ROOT.TFile(file_out_name,"recreate")
    gr_sigma_all = ROOT.TGraphErrors()
    gr_sigma_all = ph.NameIt(gr_sigma_all,name=f'sigma_all',title=f'sigma of 3rd-dev fit',xtitle='ID',ytitle='#sigma [V]')
    gr_Vbr_all = ROOT.TGraphErrors()
    gr_Vbr_all = ph.NameIt(gr_Vbr_all,name=f'Vbr_all',title=f'Breakdown voltage of all matrix',xtitle='ID',ytitle='V_{br} [V]')
    gr_h_all = ROOT.TGraphErrors()
    gr_h_all = ph.NameIt(gr_h_all,name=f'h_all',title='hysteresis of V_{br}',xtitle='ID',ytitle='h [V]')
    gr_I_OV3V_all = ROOT.TGraphErrors()
    gr_I_OV3V_all = ph.NameIt(gr_I_OV3V_all,name=f'I_OV3V',title='I_{d} with OV = 3V',xtitle='ID',ytitle='I_{d} (OV=3V) [mA]')
    gr_I_OV4V_all = ROOT.TGraphErrors()
    gr_I_OV4V_all = ph.NameIt(gr_I_OV4V_all,name=f'I_OV4V',title='I_{d} with OV = 4V',xtitle='ID',ytitle='I_{d} (OV=4V) [mA]')
    gr_I_HV55V_all = ROOT.TGraphErrors()
    gr_I_HV55V_all = ph.NameIt(gr_I_HV55V_all,name=f'I_HV55V',title='I_{d} with HV = 55V',xtitle='ID',ytitle='I_{d} (HV=55V) [mA]')
    for imatrix in matrixID_all:
        print(f"analysis matrix {imatrix}")
        #* analyze the reverse I-V curve
        gr_sigma,gr_Vbr,gr_h,gr_I_OV2V,gr_I_OV3V,gr_I_OV4V,gr_I_HV55V = do_ana_single_reverse(dir_in,imatrix)
        dir_tmp= f_out.mkdir(f"Matrix{imatrix}/")
        dir_tmp.cd()
        gr_sigma.Write()
        gr_Vbr.Write()
        gr_h.Write()
        gr_I_OV2V.Write()
        gr_I_OV3V.Write()
        gr_I_OV4V.Write()
        gr_I_HV55V.Write()
        root_h.merge_tgraph(gr_sigma,gr_sigma_all,imatrix*20,True)
        root_h.merge_tgraph(gr_Vbr,gr_Vbr_all,imatrix*20,True)
        root_h.merge_tgraph(gr_h,gr_h_all,imatrix*20,True)
        root_h.merge_tgraph(gr_I_OV3V,gr_I_OV3V_all,imatrix*20,True)
        root_h.merge_tgraph(gr_I_OV4V,gr_I_OV4V_all,imatrix*20,True)
        root_h.merge_tgraph(gr_I_HV55V,gr_I_HV55V_all,imatrix*20,True)

    f_out.cd()
    gr_sigma_all.Write()
    gr_Vbr_all.Write()
    gr_h_all.Write()
    gr_I_OV3V_all.Write()
    gr_I_OV4V_all.Write()
    gr_I_HV55V_all.Write()

    # * add channel mapping, T and dose
    dose_min = 0.18 # !this is a rough value only used for fit
    dose_max = 1.57  # unit 10^11 n_eq/cm^2
    gr_mapping      = get_obj("/media/newhd/work/PhD_thesis_related/irradation/test_in_lab/IV_scan/ch_mapping_to_sim.root","IV_scan_ch_mapping_to_sim")
    gr_dose_norm    = get_obj("/media/newhd/work/PhD_thesis_related/irradation/test_in_lab/simulation/testbeam_202104/SiPM_Dose_rate_distribution_along_chID.root","tile_SiPM_dose_vs_channel")
    gr_dose         = ROOT.TGraph(gr_dose_norm.GetN(),np.asarray(gr_dose_norm.GetX()),np.asarray(gr_dose_norm.GetY())*dose_max)
    gr_dose         = ph.NameIt(gr_dose,name=f'tile_SiPM_dose',title=f'Dose in simulation',ytitle='NIEL [10^{11} n_{eq}/cm^{2}]',xtitle='Sensor ID')
    gr_T            = get_obj("/media/newhd/work/PhD_thesis_related/irradation/test_in_lab/IV_scan/T_overall/T_overall.root","Graph0")
    gr_T.SetName("T_vs_matrixID")
    f_out.cd()
    gr_mapping.Write()
    gr_T.Write()
    gr_dose.Write()
    #TODO plot parameters vs. dose
    gr_sigma_vs_dose    = IV_ana.mapping_ch_to_dose(gr_sigma_all,gr_dose,gr_mapping)
    gr_sigma_vs_dose    = ph.NameIt(gr_sigma_vs_dose,name=f'Sigma_vs_dose',title=f'#sigma vs. dose',xtitle='NIEL [10^{11} n_{eq}/cm^{2}]',ytitle='#sigma [V]')
    gr_Vbr_vs_dose      = IV_ana.mapping_ch_to_dose(gr_Vbr_all,gr_dose,gr_mapping)
    gr_Vbr_vs_dose      = ph.NameIt(gr_Vbr_vs_dose,name=f'Vbr_vs_dose',title='V_{br} vs. dose',xtitle='NIEL [10^{11} n_{eq}/cm^{2}]',ytitle='V_{br} [V]')
    gr_h_vs_dose        = IV_ana.mapping_ch_to_dose(gr_h_all,gr_dose,gr_mapping)
    gr_h_vs_dose        = ph.NameIt(gr_h_vs_dose,name=f'h_vs_dose',title='h vs. dose',xtitle='NIEL [10^{11} n_{eq}/cm^{2}]',ytitle='h [V]')
    gr_I_OV3V_vs_dose   = IV_ana.mapping_ch_to_dose(gr_I_OV3V_all,gr_dose,gr_mapping)
    gr_I_OV3V_vs_dose   = ph.NameIt(gr_I_OV3V_vs_dose,name=f'I_OV3V_vs_dose',title='I_{d} (OV=3V) vs. dose',xtitle='NIEL [10^{11} n_{eq}/cm^{2}]',ytitle='I_{d} [mA]')
    gr_I_OV4V_vs_dose   = IV_ana.mapping_ch_to_dose(gr_I_OV4V_all,gr_dose,gr_mapping)
    gr_I_OV4V_vs_dose   = ph.NameIt(gr_I_OV4V_vs_dose,name=f'I_OV4V_vs_dose',title='I_{d} (OV=4V) vs. dose',xtitle='NIEL [10^{11} n_{eq}/cm^{2}]',ytitle='I_{d} [mA]')
    gr_I_HV55V_vs_dose   = IV_ana.mapping_ch_to_dose(gr_I_HV55V_all,gr_dose,gr_mapping)
    gr_I_HV55V_vs_dose   = ph.NameIt(gr_I_HV55V_vs_dose,name=f'I_HV55V_vs_dose',title='I_{d} (HV=55V) vs. dose',xtitle='NIEL [10^{11} n_{eq}/cm^{2}]',ytitle='I_{d} [mA]')
    
    IV_ana.constant_fit(gr_sigma_vs_dose,x_low=dose_min, x_high=dose_max,par_name="#bar{#sigma}")
    IV_ana.constant_fit(gr_Vbr_vs_dose,x_low=dose_min, x_high=dose_max,par_name="#bar{V_{br}}")
    IV_ana.constant_fit(gr_h_vs_dose,x_low=dose_min, x_high=dose_max,par_name="#bar{h}")
    pars,parEs,chi2 = IV_ana.lin_fit(gr_I_OV3V_vs_dose,x_low=dose_min, x_high=dose_max,par_name1="Slope",par_name2="Y_{offset}")
    pars,parEs,chi2 = IV_ana.lin_fit(gr_I_OV4V_vs_dose,x_low=dose_min, x_high=dose_max,par_name1="Slope",par_name2="Y_{offset}")
    pars,parEs,chi2 = IV_ana.lin_fit(gr_I_HV55V_vs_dose,x_low=dose_min, x_high=dose_max,par_name1="Slope",par_name2="Y_{offset}")
    
    dir_vs_dose= f_out.mkdir(f"diff_dose/")
    dir_vs_dose.cd()
    gr_sigma_vs_dose   .Write()
    gr_Vbr_vs_dose     .Write()
    gr_h_vs_dose       .Write()
    gr_I_OV3V_vs_dose  .Write()
    gr_I_OV4V_vs_dose  .Write()
    gr_I_HV55V_vs_dose  .Write()
    
    f_out.Close()

def do_forward_IV_ana(dir_in,file_out_name):
    matrixID_all = np.array([1,4,5,6,7])
    R_constant = 1000       #* this is the R used for current limitation
    R_constant_error = 10   #* 1% error of resistor: 1000*1% = 10
    f_out = ROOT.TFile(file_out_name,"recreate")
    gr_R_all    = ROOT.TGraphErrors()
    gr_R_all    = ph.NameIt(gr_R_all,name=f'Resistor',title=f'Resistor from different matrix',xtitle='ID',ytitle='R [#Omega]')
    gr_Rq_all    = ROOT.TGraphErrors()
    gr_Rq_all    = ph.NameIt(gr_Rq_all,name=f'Quenching_resistor',title=f'Quenching Resistor for SiPM pixel',xtitle='ID',ytitle='R_{q} [k#Omega]')
    gr_Vf_T_coef = ROOT.TGraphErrors()
    gr_Vf_T_coef = ph.NameIt(gr_Vf_T_coef,name=f'Vf_T_coef',title='V_{f} temperature coefficient',xtitle='ID',ytitle='#frac{dV_{f}}{dT} [V/#circC]')
    gr_Vf_offset = ROOT.TGraphErrors()
    gr_Vf_offset = ph.NameIt(gr_Vf_offset,name=f'Vf_offset',title='V_{f} (T=0#circC)',xtitle='ID',ytitle='V_{f}[T=0#circC]')
    gr_Vf_chi2 = ROOT.TGraphErrors()
    gr_Vf_chi2 = ph.NameIt(gr_Vf_chi2,name=f'Vf_fit_chi2',title='#chi^{2}/NDF of V_{f} vs. T fitting',xtitle='ID',ytitle='#chi^{2}/NDF')
    h_R_all = ROOT.TH1D("h_R","Resistor",500,750.,1250.)
    h_R_all = ph.NameIt(h_R_all,xtitle='R [#Omega]',ytitle='Entries')
    h_Rq_all = ROOT.TH1D("h_Rq","Quenching resistor",100,100.,400.)
    h_Rq_all = ph.NameIt(h_Rq_all,xtitle='R_{q} [k#Omega]',ytitle='Entries')
    h_Vf_T_coef = ROOT.TH1D("h_Vf_T_coef","Coefficient between V_{f} and T",100,-3.,-1.)
    h_Vf_T_coef = ph.NameIt(h_Vf_T_coef,xtitle='#frac{dV_{f}}{dT} [mV/#circC]',ytitle='Entries')
    h_Vf_offset = ROOT.TH1D("h_Vf_offset","V_{f} (T=0#circC)",100,0.,1.)
    h_Vf_offset = ph.NameIt(h_Vf_offset,xtitle='V_{f} (T=0#circC) [V]',ytitle='Entries')
    for imatrix in matrixID_all:
        for stat_annealing in range(1): #2023.03.30 fix this to 0 and for the pure IV scan
            print(f"analysis matrix {imatrix}: annealing: {stat_annealing}")
            #if (imatrix==5 and stat_annealing==0) or (imatrix==7 and stat_annealing==1):
            #    continue
            #analyze the forward I-V curve
            ID  = imatrix*2+stat_annealing
            gr_R, h_R, gr_V_forward, h_V_forward,gr_chi2, grs_scale = do_ana_single_forward(dir_in,imatrix,stat_annealing)
            # *save data in
            dir_tmp= f_out.mkdir(f"Matrix{imatrix}_annealing_stat_{stat_annealing}/")
            Vf_T_pars,Vf_T_parerrors,Vf_T_chi2 = IV_ana.lin_fit(gr_V_forward,np.min(np.asarray(gr_V_forward.GetX()))-0.5,np.max(np.asarray(gr_V_forward.GetX()))+0.5)
            h_Vf_T_coef.Fill(Vf_T_pars[0]*1000.)    #unit mV/V
            gr_Vf_T_coef.SetPoint(gr_Vf_T_coef.GetN(),ID,Vf_T_pars[0]*1000.)    #unit mV/V
            gr_Vf_T_coef.SetPointError(gr_Vf_T_coef.GetN()-1,0,Vf_T_parerrors[0]*1000.) #unit mV/C
            gr_Vf_offset.SetPoint(gr_Vf_offset.GetN(),ID,Vf_T_pars[1])
            gr_Vf_offset.SetPointError(gr_Vf_offset.GetN()-1,0,Vf_T_parerrors[1])
            h_Vf_offset.Fill(Vf_T_pars[1])
            gr_Vf_chi2.SetPoint(gr_Vf_chi2.GetN(),ID,Vf_T_chi2)
            dir_tmp.cd()
            gr_R.Write()
            h_R.Write()
            gr_V_forward.Write()
            h_V_forward.Write()
            gr_chi2.Write()
            root_h.write_grs(grs_scale,dir_tmp)

            gr_R_all.SetPoint(gr_R_all.GetN(),ID,h_R.GetMean())
            gr_R_all.SetPointError(gr_R_all.GetN()-1,0,h_R.GetRMS())
            R_q,R_q_error = cal_Rq(h_R.GetMean(),h_R.GetRMS(),R_constant,R_constant_error)
            gr_Rq_all.SetPoint(gr_Rq_all.GetN(),ID,R_q/1000.)
            gr_Rq_all.SetPointError(gr_Rq_all.GetN()-1,0,R_q_error/1000.)
            h_R_all.Fill(h_R.GetMean())
            h_Rq_all.Fill(R_q/1000.)
    IV_ana.constant_fit(gr_Vf_T_coef,np.min(np.asarray(gr_Vf_T_coef.GetX()))-0.5,np.max(np.asarray(gr_Vf_T_coef.GetX()))+0.5)
    print("DEBUG:11")
    f_out.cd()
    gr_R_all.Write()
    gr_Rq_all.Write()
    h_Rq_all.Write()
    h_R_all.Write()
    gr_Vf_T_coef.Write()
    gr_Vf_offset.Write()
    h_Vf_offset.Write()
    gr_Vf_chi2.Write()
    h_Vf_T_coef.Write()
    f_out.Close()
def main():
    if(len(sys.argv)!=3):
        print(bc.ERROR+"Usage: ",sys.argv[0], " [dir_name] [output file name w.o. postfix] "+bc.ENDC)
        exit()
    dir_in = sys.argv[1]
    file_name = sys.argv[2]
    #dir_in = "data/Wafer_at_2025_06_05-15_06_18"
    #file_name = "IV_scan_run1_matrix1_UID5"
    ## * do forward IV curve analysis
    file_name_forward = dir_in+file_name+"_forward.root"
    #2023.03.30 Zhong
    # modify the forward analysis code to the do normal IV scan: data from /media/newhd/work/PhD_thesis_related/irradation/test_in_lab/IV_scan/
    #do_forward_IV_ana(dir_in,file_name_forward)
    ## * do reverse IV curve analysis
    file_name_reverses = dir_in+file_name+"_reverse.root"
    do_reverse_IV_ana(dir_in,file_name_reverses)
    exit()

if __name__ =="__main__":
    main()


Initiated an instance of the class bcolors!


OSError: Failed to open file -f/home/user/.local/share/jupyter/runtime/kernel-1698321e-672a-448f-a27e-124adca38f39.json_reverse.root

SysError in <TFile::TFile>: file /home/user/git-repos/waferprober/-f/home/user/.local/share/jupyter/runtime/kernel-1698321e-672a-448f-a27e-124adca38f39.json_reverse.root can not be opened No such file or directory


In [ ]:
import csv
import json 
import custom_filehandler as cfile
import os


json_data = [["Voltages",  [
    0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5,
    5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5,
    10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5,
    15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5,
    20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5,
    25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0, 28.5, 29.0, 29.5,
    30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5,
    35.0, 35.5, 36.0, 36.5, 37.0, 37.5, 38.0, 38.5, 39.0, 39.5,
    40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5,
    45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5,
    50.0
]]]


files = cfile.get_all_filenames_in_folder(startdir="wafertest/IV_wafer", file_extension=".csv")
for file in files:
    measurements =[]
    line = [f"{file}", measurements]
    
    with open(file, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        for row in csv_reader:
            frow = row["Current (A)"]
            measurements.append(frow)
        json_data.append(line)
    

dump = json.dumps(json_data, indent=4)

with open("json/dump.json", mode="a", encoding="utf-8") as write_file:
    json.dump(json_data, write_file, sort_keys=False, indent=4)

In [ ]:
cfile.get_all_filenames_in_folder(startdir="/", file_extension=".csv")